In [2]:
import praw
import pandas as pd
from praw.models import MoreComments
import re

In [3]:
# Read-only instance
reddit_read_only = praw.Reddit(client_id="K85pc4mBX5iYnRsUF-iEdQ",		 # your client id
							client_secret="cDSwL4YAtDTkQsX9Risd5wnBafORGw",	 # your client secret
							user_agent="idk")	 # your user agent

subreddit = reddit_read_only.subreddit("berkeley")
#posts = subreddit.top(limit=20)

In [4]:
posts = subreddit.top(time_filter="year", limit=500)
# Scraping the top posts of the current year
 
posts_dict = {"Title": [], "Post Text": [], "Score": [],
              "Comments": [], "EECS": []
              }

In [5]:
for post in posts:
    # Title of each post
    posts_dict["Title"].append(post.title)
     
    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)
     
    # The score of a post
    posts_dict["Score"].append(post.score)
     
    comments = []
    for top_level_comment in post.comments:
        if isinstance(top_level_comment, MoreComments):
            continue
        elif (top_level_comment.body != "[deleted]"):
            comments.append(top_level_comment.body)
        else:
            comments.append('')
    # Total number of comments inside the post
    posts_dict["Comments"].append(comments)
     

	# Flair
    if post.link_flair_text=="CS/EECS" or bool(re.search(r"\b[Cc][Ss]\s?\d+[ABCabc]?", post.title + " " + post.selftext)):
        posts_dict["EECS"].append(1)
    else:
	    posts_dict["EECS"].append(0)
posts_dict

{'Title': ['URGENT: MISSING STUDENT. PLEASE LET US KNOW IF YOU HAVE SEEN HIM.',
  "standing ovation for Professor Hilfinger's last lecture after 40 years of teaching",
  'Saw Christopher Nolan on campus today possibly scouting for his new film Oppenheimer.',
  'CAL WINS 124TH BIG GAME 41-11',
  'Had some free time before the semester started, so I made a Berkeley slander video',
  'aerial view of the glade on 4/20/22 @ 4:20PM',
  'ok now that’s just silly',
  "Berkeley book fair today. Can I convince anyone to go buy my dad's book? I'll reimburse you for it.",
  'I built the Campanile out of legos!',
  'to all the non bay area ppl talking about mexican food',
  'it’s not thaaaat bad',
  'Update: reviewing all the crispest water on campus',
  'NIMBYs watch in horror as college town builds more housing for students',
  'Lowest Score in CS 170',
  'Intersections of Berkeley',
  'I think my GSI is flirting with me...',
  'Things are really hard for current bears. Who needs a Costco trip.',

In [6]:
# Saving the data in a pandas dataframe
top_posts = pd.DataFrame(posts_dict)
eecs = top_posts.loc[top_posts['EECS'] == 1]
no = top_posts.loc[top_posts['EECS'] == 0]
top_posts


,Title,Post Text,Score,Comments,EECS
0,URGENT: MISSING STUDENT. PLEASE LET US KNOW IF...,"Hello, our friend Luca Elghanayan has been mis...",2495,"[He is gone. He was my friend. Rest in peace.,...",0
1,standing ovation for Professor Hilfinger's las...,,1364,[Shame I had office hours for students; anyway...,1
2,Saw Christopher Nolan on campus today possibly...,,1232,[this is so sick. gotta casually stroll around...,0
3,CAL WINS 124TH BIG GAME 41-11,,951,"[Garbers undefeated at Stanford Stadium, GO BE...",0
4,Had some free time before the semester started...,,938,"[Lol, I wish I had an award to give this, have...",0
...,...,...,...,...,...
495,got the lowest score on the cs170 midterm,how do i academically recover from this...,216,"[notice how u said academically, not mentally....",1
496,Can we build housing here?,,216,"[Unit 1.5, Hear me out- student housing with f...",0
497,petition for landlords to replace ancient and ...,anyone else have this problem? it’s especially...,219,[I have to wear a jacket inside and use a mini...,0
498,why is this sub slowly turning into just an an...,used to be like 1 in 30 posts and they'd get d...,219,[[always has been](https://imgflip.com/s/meme/...,0


In [7]:
import cohere
from cohere.classify import Example

In [56]:
eecs["Comments"].tolist()


[['Shame I had office hours for students; anyway, happy retirement, and all my best wishes for Prof. Hilfinger!',
  'Best wishes for professor during his retirement! Thank you for teaching the class which made me love coding!',
  'Hopefully the modern era of academia means that future greats of teaching have their lectures recorded for all those in the future to reference.\n\nCongratulations to Hilfinger for a long and illustrious career, from myself and all the other students whose life he has enriched!',
  'What were his last words? "RTFM"?',
  'Is there a recording of it anywhere?',
  'Gitlet\nOf\nAll\nTime',
  'Former CS164 TA here. Loved working with Prof. Hilfinger. It was a very rewarding experience. I’m sure his students feel the same.',
  'My dad and I both had him during undergrad, 36 years apart. Truly the end of an era. Happy retirement to him!',
  'He finally retired?  Good for him!\n\nIs there any prof to take his place as "holy shit you\'ll learn a lot if you take it fro

In [8]:
from praw.reddit import re


#apiKey = 'a5QDa98c2KhjCtdXkrC5yyxcCPxcnQneFgIuycIJ'
apiKey = '5HtwM7zol4KLjnUtb6I49s98eSfTpXMQ0f5cXHdr'
co = cohere.Client(apiKey)
bleh = []
comments = [c for c in eecs["Comments"].tolist() if c != []]
for i in range(len(comments)):
    inpts = [s for s in comments[i] if s != '']
    response = co.classify(model='cohere-toxicity', inputs=inpts)
    num_tox = 0;
    for i in range(len(inpts)): 
        if response.classifications[i].__getattribute__('prediction') == 'TOXIC':
            num_tox += 1
    if num_tox/len(inpts) >= 0.85:
        bleh.append('LEAGUE TOXIC')
    elif num_tox/len(inpts) >= 0.5:
        bleh.append('TOXIC')
    else:
        bleh.append('NOT TOXIC')

print('The labels are: {}'.format(bleh))

The labels are: ['NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC', 'NOT TOXIC']


In [13]:
bleh2 = []
comments = [c for c in no["Comments"].tolist() if c != []]
for i in range(len(comments)):
    inpts = [s for s in comments[i] if s != '']
    response = co.classify(model='cohere-toxicity', inputs=inpts)
    num_tox = 0;
    for i in range(len(inpts)): 
        if response.classifications[i].__getattribute__('prediction') == 'TOXIC':
            num_tox += 1
    if num_tox/len(inpts) >= 0.85:
        bleh2.append('LEAGUE TOXIC')
    elif num_tox/len(inpts) >= 0.5:
        bleh2.append('TOXIC')
    else:
        bleh2.append('NOT TOXIC')

ConnectionError: HTTPSConnectionPool(host='api.cohere.ai', port=443): Max retries exceeded with url: /classify (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff560ee1df0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [11]:
print("Percent of Non-CS/EECS posts that are classified as Toxic: ")
print(sum([bleh2[i] == "TOXIC" or bleh2[i] == 'LEAGUE TOXIC' for i in range(len(bleh2))]) / len(bleh2))
print('\n')
print("Percent of CS/EECS posts that are classified as Toxic: ")
print(sum([bleh[i] == "TOXIC" or bleh2[i] == 'LEAGUE TOXIC' for i in range(len(bleh))]) / len(bleh))
print("Percent of Total Posts Classified as Toxic: ")
print((sum([bleh2[i] == "TOXIC" for i in range(len(bleh2))]) + sum([bleh[i] == "TOXIC" for i in range(len(bleh))]))/(len(bleh2)+len(bleh)))

Percent of Non-CS/EECS posts that are classified as Toxic: 
0.02


Percent of CS/EECS posts that are classified as Toxic: 
0.02
Percent of Total Posts Classified as Toxic: 
0.02
